# Tables OmniParser
This notebook is meant for testing and development of an "OmniParser" that can parse all components of an HTML document, including the title, captions, sentences, tables, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [27]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser
doc_parser = HTMLParser(path='data/diseases/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser)

In [28]:
# def parse_tag(tag):
#     if tag.name is not None:
#         print tag.name
#         print "---"
#         for child in tag.children:
#             parse_tag(child)

# from bs4 import BeautifulSoup
# with open('data/diseases/diseases.xhtml','r') as f:
#     soup = BeautifulSoup(f, 'lxml')
# for tag in soup.children:
#     parse_tag(tag)


In [29]:
%time corpus = cp.parse_corpus(name='Diseases Corpus')

NON-TABLE:Types of viruses, coughs, and colds
NON-TABLE:Brain Cancer
NON-TABLE:See Table Below.
NON-TABLE:Common Ailments
TABLE!
TABLE!
CPU times: user 95.8 ms, sys: 7.79 ms, total: 104 ms
Wall time: 137 ms


In [30]:
print len(corpus.documents)
print len(corpus.documents[0].tables)
print len(corpus.documents[0].phrases)
print len(corpus.documents[0].sentences)
for sent in corpus.documents[0].sentences: print sent

1
2
25
29
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Types of viruses, coughs, and colds')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Brain Cancer')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'See Table Below.')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Common Ailments')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Disease')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Location')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Year')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'Polio')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'New York')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u'1914')
Sentence(Document('diseases', Corpus (Diseases Corpus)), 0, u"I don't like Chicken Pox.")
Sentence(Document('diseases', Corpus (Diseases Corpus)), 1, u'The plague is also bad.')
Sentence(Document('diseases', Corpus (Diseases C

In [5]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


In [ ]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
table_ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

In [ ]:
# With old Candidates object:
from snorkel.candidates import EntityExtractor
ce = EntityExtractor(table_ngrams, disease_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')

for cand in candidates: print cand

In [ ]:
c = candidates[0]
for ngram in c.row_ngrams('words'): print ngram

In [ ]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)

In [ ]:
featurizer.get_features_by_candidate(candidates[1])[:]

The end.